# 📊 Power BI Integration - Hướng Dẫn Chi Tiết

**NYC Taxi Data Engineering Project - Microsoft Fabric**

Notebook này tập trung vào:
1. **Tạo Aggregated Tables** cho Power BI
2. **Hướng dẫn tạo Semantic Model** step-by-step
3. **Cấu hình Relationships** chi tiết
4. **Thư viện DAX Measures** đầy đủ
5. **Thiết kế 6 Report Pages** với từng visual cụ thể

## 1. Tạo Aggregated Tables cho Power BI

In [ ]:
from pyspark.sql import functions as F

print("📊 Verifying Gold Layer tables...")
print("=" * 60)

# Verify tables exist
tables = ["gold_fact_trips", "gold_dim_date", "gold_dim_time", 
          "gold_dim_location", "gold_dim_weather", "gold_dim_vendor",
          "gold_dim_payment_type", "gold_dim_rate_code"]

for table in tables:
    try:
        count = spark.table(table).count()
        print(f"   ✅ {table:<30} {count:>12,} rows")
    except:
        print(f"   ❌ {table:<30} NOT FOUND")

### 1.1 Daily Trends Table

In [ ]:
print("📊 Creating pbi_daily_trends...")

spark.sql("""
CREATE OR REPLACE TABLE pbi_daily_trends AS
SELECT
    d.date_sk,
    d.full_date,
    d.year,
    d.quarter,
    d.month,
    d.month_name,
    d.week_of_year,
    d.day_of_month,
    d.day_of_week,
    d.day_name,
    d.is_weekend,
    d.is_holiday,
    d.holiday_name,
    w.weather_condition,
    w.temp_avg,
    w.temp_range,
    w.precipitation,
    w.is_severe_weather,
    COUNT(*) as trip_count,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(SUM(f.fare_amount), 2) as total_fare,
    ROUND(SUM(f.tip_amount), 2) as total_tips,
    ROUND(AVG(f.fare_amount), 2) as avg_fare,
    ROUND(AVG(f.trip_distance_miles), 2) as avg_distance,
    ROUND(AVG(f.trip_duration_minutes), 2) as avg_duration,
    ROUND(AVG(f.tip_percentage), 2) as avg_tip_pct,
    SUM(CAST(f.passenger_count AS BIGINT)) as total_passengers,
    SUM(CASE WHEN f.is_rush_hour THEN 1 ELSE 0 END) as rush_hour_trips,
    SUM(CASE WHEN f.is_weekend THEN 1 ELSE 0 END) as weekend_trips
FROM gold_fact_trips f
JOIN gold_dim_date d ON f.date_sk = d.date_sk
LEFT JOIN gold_dim_weather w ON f.weather_sk = w.weather_sk
GROUP BY 
    d.date_sk, d.full_date, d.year, d.quarter, d.month, d.month_name,
    d.week_of_year, d.day_of_month, d.day_of_week, d.day_name,
    d.is_weekend, d.is_holiday, d.holiday_name,
    w.weather_condition, w.temp_avg, w.temp_range, w.precipitation, w.is_severe_weather
""")

print(f"✅ pbi_daily_trends: {spark.table('pbi_daily_trends').count():,} rows")

### 1.2 Hourly Patterns Table

In [ ]:
print("📊 Creating pbi_hourly_patterns...")

spark.sql("""
CREATE OR REPLACE TABLE pbi_hourly_patterns AS
SELECT
    t.hour_24 as hour,
    t.hour_12,
    t.am_pm,
    t.time_of_day,
    t.peak_period,
    d.day_of_week,
    d.day_name,
    d.is_weekend,
    COUNT(*) as trip_count,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.fare_amount), 2) as avg_fare,
    ROUND(AVG(f.trip_distance_miles), 2) as avg_distance,
    ROUND(AVG(f.trip_duration_minutes), 2) as avg_duration,
    ROUND(AVG(f.tip_percentage), 2) as avg_tip_pct,
    ROUND(AVG(f.avg_speed_mph), 2) as avg_speed
FROM gold_fact_trips f
JOIN gold_dim_time t ON f.time_sk = t.time_sk
JOIN gold_dim_date d ON f.date_sk = d.date_sk
GROUP BY 
    t.hour_24, t.hour_12, t.am_pm, t.time_of_day, t.peak_period,
    d.day_of_week, d.day_name, d.is_weekend
""")

print(f"✅ pbi_hourly_patterns: {spark.table('pbi_hourly_patterns').count():,} rows")

### 1.3 Location Analytics Table

In [ ]:
print("📊 Creating pbi_location_analytics...")

spark.sql("""
CREATE OR REPLACE TABLE pbi_location_analytics AS
SELECT
    pu.location_id as pickup_location_id,
    pu.zone_name as pickup_zone,
    pu.borough as pickup_borough,
    pu.service_zone as pickup_service_zone,
    pu.is_airport as pickup_is_airport,
    do.location_id as dropoff_location_id,
    do.zone_name as dropoff_zone,
    do.borough as dropoff_borough,
    do.is_airport as dropoff_is_airport,
    COUNT(*) as trip_count,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(SUM(f.tip_amount), 2) as total_tips,
    ROUND(AVG(f.fare_amount), 2) as avg_fare,
    ROUND(AVG(f.trip_distance_miles), 2) as avg_distance,
    ROUND(AVG(f.trip_duration_minutes), 2) as avg_duration,
    ROUND(AVG(f.tip_percentage), 2) as avg_tip_pct
FROM gold_fact_trips f
JOIN gold_dim_location pu ON f.pickup_location_sk = pu.location_sk
JOIN gold_dim_location do ON f.dropoff_location_sk = do.location_sk
GROUP BY 
    pu.location_id, pu.zone_name, pu.borough, pu.service_zone, pu.is_airport,
    do.location_id, do.zone_name, do.borough, do.is_airport
""")

print(f"✅ pbi_location_analytics: {spark.table('pbi_location_analytics').count():,} rows")

### 1.4 Weather Impact Table

In [ ]:
print("📊 Creating pbi_weather_impact...")

spark.sql("""
CREATE OR REPLACE TABLE pbi_weather_impact AS
SELECT
    w.weather_sk,
    w.weather_condition,
    w.temp_range,
    w.temp_avg,
    w.precipitation,
    w.snowfall,
    w.is_severe_weather,
    w.weather_score,
    d.year,
    d.month,
    d.month_name,
    d.is_weekend,
    COUNT(*) as trip_count,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.fare_amount), 2) as avg_fare,
    ROUND(AVG(f.trip_distance_miles), 2) as avg_distance,
    ROUND(AVG(f.trip_duration_minutes), 2) as avg_duration,
    ROUND(AVG(f.avg_speed_mph), 2) as avg_speed,
    ROUND(AVG(f.tip_percentage), 2) as avg_tip_pct
FROM gold_fact_trips f
JOIN gold_dim_weather w ON f.weather_sk = w.weather_sk
JOIN gold_dim_date d ON f.date_sk = d.date_sk
GROUP BY 
    w.weather_sk, w.weather_condition, w.temp_range, w.temp_avg, 
    w.precipitation, w.snowfall, w.is_severe_weather, w.weather_score,
    d.year, d.month, d.month_name, d.is_weekend
""")

print(f"✅ pbi_weather_impact: {spark.table('pbi_weather_impact').count():,} rows")

### 1.5 Payment Analysis Table

In [ ]:
print("📊 Creating pbi_payment_analysis...")

spark.sql("""
CREATE OR REPLACE TABLE pbi_payment_analysis AS
SELECT
    p.payment_type_sk,
    p.payment_type_id,
    p.payment_type_name,
    p.payment_category,
    p.allows_tip,
    d.year,
    d.month,
    d.month_name,
    d.quarter,
    COUNT(*) as trip_count,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(SUM(f.fare_amount), 2) as total_fare,
    ROUND(SUM(f.tip_amount), 2) as total_tips,
    ROUND(AVG(f.fare_amount), 2) as avg_fare,
    ROUND(AVG(f.tip_percentage), 2) as avg_tip_pct,
    ROUND(AVG(f.trip_distance_miles), 2) as avg_distance
FROM gold_fact_trips f
JOIN gold_dim_payment_type p ON f.payment_type_sk = p.payment_type_sk
JOIN gold_dim_date d ON f.date_sk = d.date_sk
GROUP BY 
    p.payment_type_sk, p.payment_type_id, p.payment_type_name, 
    p.payment_category, p.allows_tip,
    d.year, d.month, d.month_name, d.quarter
""")

print(f"✅ pbi_payment_analysis: {spark.table('pbi_payment_analysis').count():,} rows")

### 1.6 Borough Monthly Summary

In [ ]:
print("📊 Creating pbi_borough_monthly...")

spark.sql("""
CREATE OR REPLACE TABLE pbi_borough_monthly AS
SELECT
    d.year,
    d.month,
    d.month_name,
    d.quarter,
    pu.borough as pickup_borough,
    do.borough as dropoff_borough,
    COUNT(*) as trip_count,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(SUM(f.tip_amount), 2) as total_tips,
    ROUND(AVG(f.fare_amount), 2) as avg_fare,
    ROUND(AVG(f.trip_distance_miles), 2) as avg_distance,
    ROUND(AVG(f.tip_percentage), 2) as avg_tip_pct,
    SUM(CASE WHEN f.is_rush_hour THEN 1 ELSE 0 END) as rush_hour_trips,
    SUM(CAST(f.passenger_count AS BIGINT)) as total_passengers
FROM gold_fact_trips f
JOIN gold_dim_date d ON f.date_sk = d.date_sk
JOIN gold_dim_location pu ON f.pickup_location_sk = pu.location_sk
JOIN gold_dim_location do ON f.dropoff_location_sk = do.location_sk
GROUP BY 
    d.year, d.month, d.month_name, d.quarter,
    pu.borough, do.borough
""")

print(f"✅ pbi_borough_monthly: {spark.table('pbi_borough_monthly').count():,} rows")

### 1.7 Summary - Tables Created

In [ ]:
print("=" * 70)
print("📊 TẤT CẢ TABLES CHO POWER BI")
print("=" * 70)

pbi_tables = [
    ("pbi_daily_trends", "Xu hướng theo ngày"),
    ("pbi_hourly_patterns", "Patterns theo giờ"),
    ("pbi_location_analytics", "Phân tích theo vùng"),
    ("pbi_weather_impact", "Ảnh hưởng thời tiết"),
    ("pbi_payment_analysis", "Phân tích thanh toán"),
    ("pbi_borough_monthly", "Borough theo tháng")
]

print(f"\n{'Table':<30} {'Rows':>12}   Mô tả")
print("-" * 70)

for table, desc in pbi_tables:
    try:
        count = spark.table(table).count()
        print(f"{table:<30} {count:>12,}   {desc}")
    except:
        print(f"{table:<30} {'ERROR':>12}   {desc}")

---
# 📘 PHẦN 2: HƯỚNG DẪN TẠO SEMANTIC MODEL

## 2.1 Các bước tạo Semantic Model trong Fabric

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    BƯỚC 1: MỞ LAKEHOUSE                                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  1. Vào Microsoft Fabric: https://app.fabric.microsoft.com                   ║
║                                                                              ║
║  2. Chọn Workspace của bạn ở sidebar trái                                    ║
║                                                                              ║
║  3. Click vào Lakehouse "NYC_Taxi_Lakehouse"                                 ║
║                                                                              ║
║  4. Bạn sẽ thấy Explorer panel với:                                          ║
║     📁 Tables                                                                ║
║        ├── gold_fact_trips                                                   ║
║        ├── gold_dim_date                                                     ║
║        ├── gold_dim_time                                                     ║
║        ├── gold_dim_location                                                 ║
║        ├── gold_dim_weather                                                  ║
║        ├── pbi_daily_trends                                                  ║
║        └── ...                                                               ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════════════════╗
║                    BƯỚC 2: TẠO SEMANTIC MODEL                                ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  1. Ở góc trên phải của Lakehouse, click nút:                                ║
║     ┌─────────────────────────┐                                              ║
║     │ 📊 New semantic model   │                                              ║
║     └─────────────────────────┘                                              ║
║                                                                              ║
║  2. Popup "New semantic model" xuất hiện:                                    ║
║     ┌──────────────────────────────────────────┐                             ║
║     │ Name: NYC_Taxi_Analytics                 │                             ║
║     │                                          │                             ║
║     │ Select tables to include:                │                             ║
║     │ ☑️ gold_fact_trips                       │                             ║
║     │ ☑️ gold_dim_date                         │                             ║
║     │ ☑️ gold_dim_time                         │                             ║
║     │ ☑️ gold_dim_location                     │                             ║
║     │ ☑️ gold_dim_vendor                       │                             ║
║     │ ☑️ gold_dim_payment_type                 │                             ║
║     │ ☑️ gold_dim_rate_code                    │                             ║
║     │ ☑️ gold_dim_weather                      │                             ║
║     │ ☑️ pbi_daily_trends                      │                             ║
║     │ ☑️ pbi_hourly_patterns                   │                             ║
║     │ ☑️ pbi_location_analytics                │                             ║
║     │ ☐ pbi_weather_impact (optional)         │                             ║
║     │ ☐ pbi_payment_analysis (optional)       │                             ║
║     │                                          │                             ║
║     │              [Confirm]                   │                             ║
║     └──────────────────────────────────────────┘                             ║
║                                                                              ║
║  3. Click "Confirm" để tạo Semantic Model                                    ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
```


## 2.2 Cấu hình Relationships - Chi tiết từng bước

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    BƯỚC 3: MỞ MODEL VIEW                                     ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  1. Sau khi tạo Semantic Model, nó sẽ mở ra                                  ║
║                                                                              ║
║  2. Click tab "Model" ở bottom hoặc icon:                                    ║
║     ┌─────┐                                                                  ║
║     │ ⬡⬡⬡ │  Model view                                                     ║
║     └─────┘                                                                  ║
║                                                                              ║
║  3. Bạn sẽ thấy các tables hiển thị dạng boxes                               ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════════════════╗
║                    BƯỚC 4: TẠO RELATIONSHIPS                                 ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  CÁCH 1: Kéo thả (Drag & Drop)                                               ║
║  ─────────────────────────────                                               ║
║  1. Tìm column "date_sk" trong gold_fact_trips                               ║
║  2. Kéo nó thả vào "date_sk" trong gold_dim_date                             ║
║  3. Relationship tự động được tạo!                                           ║
║                                                                              ║
║  CÁCH 2: Manage Relationships Dialog                                         ║
║  ───────────────────────────────────                                         ║
║  1. Click "Manage relationships" ở toolbar                                   ║
║  2. Click "New relationship"                                                 ║
║  3. Chọn From table và To table                                              ║
║  4. Chọn columns                                                             ║
║  5. Click OK                                                                 ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
```


## 2.3 Danh sách Relationships cần tạo

### 📋 BẢNG RELATIONSHIPS CHI TIẾT

| # | From Table | From Column | → | To Table | To Column | Type | Active |
|---|------------|-------------|---|----------|-----------|------|--------|
| 1 | gold_fact_trips | date_sk | → | gold_dim_date | date_sk | N:1 | ✅ Yes |
| 2 | gold_fact_trips | time_sk | → | gold_dim_time | time_sk | N:1 | ✅ Yes |
| 3 | gold_fact_trips | pickup_location_sk | → | gold_dim_location | location_sk | N:1 | ✅ Yes |
| 4 | gold_fact_trips | dropoff_location_sk | → | gold_dim_location | location_sk | N:1 | ❌ No* |
| 5 | gold_fact_trips | vendor_sk | → | gold_dim_vendor | vendor_sk | N:1 | ✅ Yes |
| 6 | gold_fact_trips | payment_type_sk | → | gold_dim_payment_type | payment_type_sk | N:1 | ✅ Yes |
| 7 | gold_fact_trips | rate_code_sk | → | gold_dim_rate_code | rate_code_sk | N:1 | ✅ Yes |
| 8 | gold_fact_trips | weather_sk | → | gold_dim_weather | weather_sk | N:1 | ✅ Yes |

> **\*Lưu ý Relationship #4:** Power BI không cho phép 2 active relationships từ 1 table đến cùng 1 table khác.
> Relationship đến dropoff_location phải set **Inactive**. Dùng DAX `USERELATIONSHIP()` khi cần.


## 2.4 Cách tạo từng Relationship - Screenshots mô tả

```
╔══════════════════════════════════════════════════════════════════════════════╗
║           RELATIONSHIP 1: Fact → Date Dimension                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌─────────────────────┐                    ┌─────────────────────┐          ║
║  │  gold_fact_trips    │                    │   gold_dim_date     │          ║
║  ├─────────────────────┤                    ├─────────────────────┤          ║
║  │ trip_id             │                    │ date_sk        🔑   │          ║
║  │ date_sk         ────┼───────────────────►│ full_date           │          ║
║  │ time_sk             │       N:1          │ year                │          ║
║  │ pickup_location_sk  │                    │ month               │          ║
║  │ dropoff_location_sk │                    │ day_name            │          ║
║  │ ...                 │                    │ is_weekend          │          ║
║  └─────────────────────┘                    └─────────────────────┘          ║
║                                                                              ║
║  Settings:                                                                   ║
║  • Cardinality: Many to One (N:1)                                            ║
║  • Cross filter direction: Both                                              ║
║  • Make this relationship active: ✅ Yes                                     ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════════════════╗
║           RELATIONSHIP 3 & 4: Fact → Location (Role-Playing)                 ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌─────────────────────┐                    ┌─────────────────────┐          ║
║  │  gold_fact_trips    │                    │  gold_dim_location  │          ║
║  ├─────────────────────┤     ACTIVE        ├─────────────────────┤          ║
║  │ pickup_location_sk ─┼───────────────────►│ location_sk    🔑   │          ║
║  │                     │       N:1          │ zone_name           │          ║
║  │ dropoff_location_sk┼ - - - - - - - - - -►│ borough             │          ║
║  │                     │     INACTIVE       │ is_airport          │          ║
║  └─────────────────────┘                    └─────────────────────┘          ║
║                                                                              ║
║  Relationship 3 (Pickup - ACTIVE):                                           ║
║  • Make this relationship active: ✅ Yes                                     ║
║                                                                              ║
║  Relationship 4 (Dropoff - INACTIVE):                                        ║
║  • Make this relationship active: ❌ No                                      ║
║  • Dùng USERELATIONSHIP() trong DAX khi cần query dropoff                    ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
```


## 2.5 Star Schema Diagram hoàn chỉnh

```
                                   ┌─────────────────┐
                                   │  gold_dim_date  │
                                   │    (date_sk)    │
                                   │  • full_date    │
                                   │  • year, month  │
                                   │  • day_name     │
                                   │  • is_weekend   │
                                   │  • is_holiday   │
                                   └────────┬────────┘
                                            │
                                            │ N:1
     ┌─────────────────┐                    │                    ┌─────────────────┐
     │ gold_dim_weather│                    │                    │  gold_dim_time  │
     │  (weather_sk)   │                    │                    │   (time_sk)     │
     │ • condition     │                    │                    │  • hour_24      │
     │ • temp_avg      │                    │                    │  • time_of_day  │
     │ • precipitation │                    │                    │  • peak_period  │
     └────────┬────────┘                    │                    └────────┬────────┘
              │                             │                             │
              │ N:1                         │                         N:1 │
              │         ┌───────────────────┴───────────────────┐         │
              └────────►│           gold_fact_trips             │◄────────┘
                        │      (Central Fact Table)             │
                        │                                       │
                        │  • trip_id (PK)                       │
                        │  • date_sk (FK)                       │
                        │  • time_sk (FK)                       │
                        │  • pickup_location_sk (FK)            │
                        │  • dropoff_location_sk (FK)           │
                        │  • vendor_sk (FK)                     │
                        │  • payment_type_sk (FK)               │
                        │  • weather_sk (FK)                    │
                        │                                       │
                        │  MEASURES:                            │
                        │  • fare_amount                        │
                        │  • tip_amount                         │
                        │  • total_amount                       │
                        │  • trip_distance_miles                │
                        │  • trip_duration_minutes              │
                        └───────────────────┬───────────────────┘
                               │            │            │
                           N:1 │        N:1 │        N:1 │
                               │            │            │
                               ▼            ▼            ▼
                 ┌─────────────────┐ ┌─────────────────┐ ┌─────────────────────┐
                 │gold_dim_location│ │ gold_dim_vendor │ │gold_dim_payment_type│
                 │  (location_sk)  │ │   (vendor_sk)   │ │ (payment_type_sk)   │
                 │ • zone_name     │ │ • vendor_name   │ │ • payment_name      │
                 │ • borough       │ │ • vendor_type   │ │ • allows_tip        │
                 │ • is_airport    │ └─────────────────┘ └─────────────────────┘
                 └─────────────────┘
                       ▲
                       │ (Inactive - for dropoff)
                       │
```


---
# 📘 PHẦN 3: THƯ VIỆN DAX MEASURES

## 3.1 Cách tạo Measures trong Power BI

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    CÁCH TẠO MEASURE                                          ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  BƯỚC 1: Tạo Measures Table (Best Practice)                                  ║
║  ──────────────────────────────────────────                                  ║
║  1. Modeling tab → New table                                                 ║
║  2. Nhập: _Measures = ROW("Dummy", 1)                                        ║
║  3. Table này để chứa tất cả measures                                        ║
║                                                                              ║
║  BƯỚC 2: Tạo Measure                                                         ║
║  ─────────────────────                                                       ║
║  1. Click vào table _Measures trong Fields pane                              ║
║  2. Modeling tab → New measure                                               ║
║  3. Nhập formula DAX vào formula bar                                         ║
║  4. Press Enter                                                              ║
║                                                                              ║
║  BƯỚC 3: Format Measure                                                      ║
║  ────────────────────                                                        ║
║  1. Chọn measure vừa tạo                                                     ║
║  2. Measure tools tab → Format: Currency/Percentage/Number                   ║
║  3. Decimal places: 0, 1, 2...                                               ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
```


## 3.2 Basic Measures - Copy vào Power BI

In [ ]:
dax_basic = """
// ════════════════════════════════════════════════════════════════════════════
// BASIC METRICS
// ════════════════════════════════════════════════════════════════════════════

// 1. Total Trips - Tổng số chuyến xe
Total Trips = 
COUNTROWS(gold_fact_trips)

// 2. Total Revenue - Tổng doanh thu
Total Revenue = 
SUM(gold_fact_trips[total_amount])

// 3. Total Fare - Tổng tiền cước
Total Fare = 
SUM(gold_fact_trips[fare_amount])

// 4. Total Tips - Tổng tiền tip
Total Tips = 
SUM(gold_fact_trips[tip_amount])

// 5. Total Distance - Tổng quãng đường (miles)
Total Distance = 
SUM(gold_fact_trips[trip_distance_miles])

// 6. Total Passengers - Tổng hành khách
Total Passengers = 
SUM(gold_fact_trips[passenger_count])

// 7. Total Duration - Tổng thời gian (phút)
Total Duration = 
SUM(gold_fact_trips[trip_duration_minutes])
"""
print(dax_basic)

## 3.3 Average Measures

In [ ]:
dax_averages = """
// ════════════════════════════════════════════════════════════════════════════
// AVERAGE METRICS
// ════════════════════════════════════════════════════════════════════════════

// 8. Average Fare - Giá cước trung bình
Avg Fare = 
AVERAGE(gold_fact_trips[fare_amount])

// 9. Average Distance - Quãng đường trung bình
Avg Distance = 
AVERAGE(gold_fact_trips[trip_distance_miles])

// 10. Average Duration - Thời gian trung bình (phút)
Avg Duration = 
AVERAGE(gold_fact_trips[trip_duration_minutes])

// 11. Average Tip % - Phần trăm tip trung bình
Avg Tip % = 
AVERAGE(gold_fact_trips[tip_percentage])

// 12. Average Speed - Tốc độ trung bình (mph)
Avg Speed = 
AVERAGE(gold_fact_trips[avg_speed_mph])

// 13. Revenue per Trip - Doanh thu mỗi chuyến
Revenue per Trip = 
DIVIDE([Total Revenue], [Total Trips], 0)

// 14. Revenue per Mile - Doanh thu mỗi dặm
Revenue per Mile = 
DIVIDE([Total Revenue], [Total Distance], 0)

// 15. Trips per Day - Số chuyến mỗi ngày
Trips per Day = 
DIVIDE(
    [Total Trips],
    DISTINCTCOUNT(gold_dim_date[full_date]),
    0
)
"""
print(dax_averages)

## 3.4 Time Intelligence Measures

In [ ]:
dax_time = """
// ════════════════════════════════════════════════════════════════════════════
// TIME INTELLIGENCE
// ════════════════════════════════════════════════════════════════════════════

// 16. Revenue YTD - Doanh thu từ đầu năm
Revenue YTD = 
TOTALYTD([Total Revenue], gold_dim_date[full_date])

// 17. Revenue MTD - Doanh thu từ đầu tháng
Revenue MTD = 
TOTALMTD([Total Revenue], gold_dim_date[full_date])

// 18. Revenue QTD - Doanh thu từ đầu quý
Revenue QTD = 
TOTALQTD([Total Revenue], gold_dim_date[full_date])

// 19. Revenue Previous Year - Doanh thu năm trước
Revenue PY = 
CALCULATE(
    [Total Revenue],
    SAMEPERIODLASTYEAR(gold_dim_date[full_date])
)

// 20. Revenue Previous Month - Doanh thu tháng trước
Revenue PM = 
CALCULATE(
    [Total Revenue],
    PREVIOUSMONTH(gold_dim_date[full_date])
)

// 21. YoY Growth % - Tăng trưởng so với năm trước
YoY Growth % = 
VAR CurrentRevenue = [Total Revenue]
VAR PreviousRevenue = [Revenue PY]
RETURN
DIVIDE(CurrentRevenue - PreviousRevenue, PreviousRevenue, 0)

// 22. MoM Growth % - Tăng trưởng so với tháng trước
MoM Growth % = 
VAR CurrentRevenue = [Total Revenue]
VAR PreviousRevenue = [Revenue PM]
RETURN
DIVIDE(CurrentRevenue - PreviousRevenue, PreviousRevenue, 0)

// 23. Trips PY - Số chuyến năm trước (để so sánh)
Trips PY = 
CALCULATE(
    [Total Trips],
    SAMEPERIODLASTYEAR(gold_dim_date[full_date])
)

// 24. Trips YoY Growth % - Tăng trưởng chuyến xe YoY
Trips YoY Growth % = 
VAR CurrentTrips = [Total Trips]
VAR PreviousTrips = [Trips PY]
RETURN
DIVIDE(CurrentTrips - PreviousTrips, PreviousTrips, 0)
"""
print(dax_time)

## 3.5 Conditional/Filtered Measures

In [ ]:
dax_conditional = """
// ════════════════════════════════════════════════════════════════════════════
// CONDITIONAL / FILTERED MEASURES
// ════════════════════════════════════════════════════════════════════════════

// 25. Weekend Trips - Chuyến cuối tuần
Weekend Trips = 
CALCULATE([Total Trips], gold_fact_trips[is_weekend] = TRUE)

// 26. Weekday Trips - Chuyến ngày thường
Weekday Trips = 
CALCULATE([Total Trips], gold_fact_trips[is_weekend] = FALSE)

// 27. Weekend Revenue
Weekend Revenue = 
CALCULATE([Total Revenue], gold_fact_trips[is_weekend] = TRUE)

// 28. Rush Hour Trips - Chuyến giờ cao điểm
Rush Hour Trips = 
CALCULATE([Total Trips], gold_fact_trips[is_rush_hour] = TRUE)

// 29. Rush Hour % - Tỷ lệ giờ cao điểm
Rush Hour % = 
DIVIDE([Rush Hour Trips], [Total Trips], 0)

// 30. High Tip Trips - Chuyến tip cao (>=20%)
High Tip Trips = 
CALCULATE(
    [Total Trips],
    gold_fact_trips[tip_percentage] >= 20
)

// 31. High Tip Rate - Tỷ lệ tip cao
High Tip Rate = 
DIVIDE([High Tip Trips], [Total Trips], 0)

// 32. Credit Card Trips
Credit Card Trips = 
CALCULATE(
    [Total Trips],
    gold_dim_payment_type[payment_type_name] = "Credit Card"
)

// 33. Cash Trips
Cash Trips = 
CALCULATE(
    [Total Trips],
    gold_dim_payment_type[payment_type_name] = "Cash"
)

// 34. Airport Pickup Trips
Airport Pickup Trips = 
CALCULATE(
    [Total Trips],
    gold_dim_location[is_airport] = TRUE
)

// 35. Airport Revenue
Airport Revenue = 
CALCULATE(
    [Total Revenue],
    gold_dim_location[is_airport] = TRUE
)
"""
print(dax_conditional)

## 3.6 Dropoff Location Measures (Role-Playing Dimension)

In [ ]:
dax_dropoff = """
// ════════════════════════════════════════════════════════════════════════════
// DROPOFF LOCATION MEASURES (Using Inactive Relationship)
// ════════════════════════════════════════════════════════════════════════════

// 36. Dropoff Trips by Location
Dropoff Trips = 
CALCULATE(
    [Total Trips],
    USERELATIONSHIP(
        gold_fact_trips[dropoff_location_sk],
        gold_dim_location[location_sk]
    )
)

// 37. Dropoff Revenue
Dropoff Revenue = 
CALCULATE(
    [Total Revenue],
    USERELATIONSHIP(
        gold_fact_trips[dropoff_location_sk],
        gold_dim_location[location_sk]
    )
)

// 38. Airport Dropoff Trips
Airport Dropoff Trips = 
CALCULATE(
    [Total Trips],
    USERELATIONSHIP(
        gold_fact_trips[dropoff_location_sk],
        gold_dim_location[location_sk]
    ),
    gold_dim_location[is_airport] = TRUE
)
"""
print(dax_dropoff)

## 3.7 KPI Formatting Measures

In [ ]:
dax_formatting = """
// ════════════════════════════════════════════════════════════════════════════
// KPI FORMATTING MEASURES
// ════════════════════════════════════════════════════════════════════════════

// 39. Revenue Formatted (K/M/B)
Revenue Formatted = 
VAR Revenue = [Total Revenue]
RETURN
IF(Revenue >= 1000000000, 
    FORMAT(Revenue / 1000000000, "$#,##0.0B"),
IF(Revenue >= 1000000, 
    FORMAT(Revenue / 1000000, "$#,##0.0M"),
IF(Revenue >= 1000, 
    FORMAT(Revenue / 1000, "$#,##0.0K"),
    FORMAT(Revenue, "$#,##0"))))

// 40. Trips Formatted (K/M)
Trips Formatted = 
VAR Trips = [Total Trips]
RETURN
IF(Trips >= 1000000, 
    FORMAT(Trips / 1000000, "#,##0.0M"),
IF(Trips >= 1000, 
    FORMAT(Trips / 1000, "#,##0.0K"),
    FORMAT(Trips, "#,##0")))

// 41. YoY Arrow Indicator
YoY Indicator = 
VAR Growth = [YoY Growth %]
RETURN
IF(Growth > 0, "▲ " & FORMAT(Growth, "0.0%"),
IF(Growth < 0, "▼ " & FORMAT(Growth, "0.0%"),
"● 0.0%"))

// 42. Performance Color (for conditional formatting)
Performance Color = 
VAR Growth = [YoY Growth %]
RETURN
IF(Growth >= 0.1, "Green",
IF(Growth >= 0, "Yellow",
"Red"))
"""
print(dax_formatting)

---
# 📘 PHẦN 4: THIẾT KẾ 6 REPORT PAGES

## 4.1 Page 1: Executive Dashboard

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    🚕 NYC TAXI EXECUTIVE DASHBOARD                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌─────────────────────┐║
║  │Total Trips│ │ Revenue  │ │ Avg Fare │ │Avg Tip % │ │ Slicers:            │║
║  │   CARD   │ │   CARD   │ │   CARD   │ │   CARD   │ │ • Year              │║
║  │  185.7M  │ │  $2.5B   │ │  $12.45  │ │  18.2%   │ │ • Month             │║
║  │ ▲ +5.2%  │ │ ▲ +8.1%  │ │ ▲ +2.3%  │ │ ▼ -0.5%  │ │ • Borough           │║
║  └──────────┘ └──────────┘ └──────────┘ └──────────┘ │ • Payment Type      │║
║                                                       └─────────────────────┘║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │            📈 REVENUE & TRIPS TREND                                    │  ║
║  │            [LINE CHART - Dual Y-Axis]                                  │  ║
║  │                                                                        │  ║
║  │   X-axis: gold_dim_date[full_date]                                     │  ║
║  │   Y-axis (left): [Total Revenue]                                       │  ║
║  │   Y-axis (right): [Total Trips]                                        │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌─────────────────────────────────┐ ┌──────────────────────────────────────┐║
║  │                                 │ │                                      │║
║  │  🍩 TRIPS BY BOROUGH            │ │  📊 REVENUE BY TIME OF DAY           │║
║  │  [DONUT CHART]                  │ │  [CLUSTERED BAR CHART]               │║
║  │                                 │ │                                      │║
║  │  Legend: gold_dim_location      │ │  X-axis: gold_dim_time[time_of_day]  │║
║  │          [borough]              │ │  Y-axis: [Total Revenue]             │║
║  │  Values: [Total Trips]          │ │  Legend: gold_dim_date[is_weekend]   │║
║  │                                 │ │                                      │║
║  └─────────────────────────────────┘ └──────────────────────────────────────┘║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

HƯỚNG DẪN TẠO:

1️⃣ KPI CARDS (4 cards ở trên):
   • Insert → Card visual
   • Kéo measure [Total Trips] vào Fields
   • Format → Callout value → Display units: Auto
   • Thêm Subtitle với YoY measure

2️⃣ LINE CHART (Trend):
   • Insert → Line chart
   • X-axis: gold_dim_date[full_date]
   • Y-axis: [Total Revenue]
   • Secondary Y-axis: [Total Trips] (Format → Y-axis → Secondary axis: ON)
   • Add trend line: Analytics → Trend line

3️⃣ DONUT CHART:
   • Insert → Donut chart
   • Legend: gold_dim_location[borough]
   • Values: [Total Trips]
   • Format → Detail labels: Category, Value

4️⃣ CLUSTERED BAR:
   • Insert → Clustered bar chart
   • Y-axis: gold_dim_time[time_of_day]
   • X-axis: [Total Revenue]
   • Legend: gold_dim_date[is_weekend]

5️⃣ SLICERS:
   • Insert → Slicer
   • Field: gold_dim_date[year] - Dropdown style
   • Field: gold_dim_date[month_name] - Dropdown
   • Field: gold_dim_location[borough] - Buttons
```


## 4.2 Page 2: Geographic Analysis

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    🗺️ GEOGRAPHIC ANALYSIS                                   ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │                    🗺️ NYC PICKUP HEATMAP                               │  ║
║  │                    [FILLED MAP or SHAPE MAP]                           │  ║
║  │                                                                        │  ║
║  │   Location: gold_dim_location[borough]                                 │  ║
║  │   Color saturation: [Total Trips]                                      │  ║
║  │   Tooltips: [Total Revenue], [Avg Fare], [Avg Tip %]                   │  ║
║  │                                                                        │  ║
║  │   💡 NYC Borough coordinates sẽ tự động map                            │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌─────────────────────────────────┐ ┌──────────────────────────────────────┐║
║  │                                 │ │                                      │║
║  │  📊 TOP 10 PICKUP ZONES         │ │  📋 TOP ROUTES TABLE                 │║
║  │  [HORIZONTAL BAR CHART]         │ │  [TABLE VISUAL]                      │║
║  │                                 │ │                                      │║
║  │  Y: pbi_location_analytics      │ │  Columns from pbi_location_analytics:│║
║  │     [pickup_zone]               │ │  • pickup_zone                       │║
║  │  X: [trip_count]                │ │  • dropoff_zone                      │║
║  │                                 │ │  • trip_count                        │║
║  │  Filter: Top N = 10             │ │  • total_revenue                     │║
║  │  Sort: Descending by trip_count │ │  • avg_fare                          │║
║  │                                 │ │                                      │║
║  │  Data labels: ON                │ │  Sort: trip_count descending         │║
║  │                                 │ │  Top N filter: 20                    │║
║  └─────────────────────────────────┘ └──────────────────────────────────────┘║
║                                                                              ║
║  ┌─────────────────────────────────┐ ┌──────────────────────────────────────┐║
║  │                                 │ │                                      │║
║  │  🛫 AIRPORT vs NON-AIRPORT      │ │  📊 BOROUGH COMPARISON               │║
║  │  [PIE CHART]                    │ │  [STACKED BAR]                       │║
║  │                                 │ │                                      │║
║  │  Legend: is_airport             │ │  Y: pickup_borough                   │║
║  │  Values: trip_count             │ │  X: trip_count, total_revenue        │║
║  │                                 │ │                                      │║
║  └─────────────────────────────────┘ └──────────────────────────────────────┘║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

HƯỚNG DẪN TẠO MAP:

1️⃣ Enable Map Visual (nếu chưa có):
   • File → Options → Security → Map and filled map visuals: ON

2️⃣ Filled Map:
   • Insert → Filled map
   • Location: gold_dim_location[borough]
   • Color saturation: [Total Trips]
   • Format → Map settings → Auto zoom: ON

3️⃣ Top N Filter:
   • Trên Bar chart, vào Filters pane
   • Kéo pickup_zone vào Filters on this visual
   • Filter type: Top N
   • Show items: Top 10
   • By value: trip_count
```


## 4.3 Page 3: Time Analysis

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                         ⏰ TIME ANALYSIS                                     ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │            📈 HOURLY DEMAND PATTERN                                    │  ║
║  │            [AREA CHART]                                                │  ║
║  │                                                                        │  ║
║  │   X-axis: pbi_hourly_patterns[hour] (0-23)                             │  ║
║  │   Y-axis: SUM(pbi_hourly_patterns[trip_count])                         │  ║
║  │   Legend: pbi_hourly_patterns[is_weekend]                              │  ║
║  │                                                                        │  ║
║  │   Format: Gradient fill, Show data labels at peaks                     │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌─────────────────────────────────┐ ┌──────────────────────────────────────┐║
║  │                                 │ │                                      │║
║  │  📊 DAY OF WEEK PATTERN         │ │  📈 MONTHLY TREND BY YEAR            │║
║  │  [COLUMN CHART]                 │ │  [LINE CHART]                        │║
║  │                                 │ │                                      │║
║  │  X: pbi_hourly_patterns         │ │  X: gold_dim_date[month_name]        │║
║  │     [day_name]                  │ │  Y: [Total Revenue]                  │║
║  │  Y: trip_count                  │ │  Legend: gold_dim_date[year]         │║
║  │                                 │ │                                      │║
║  │  Sort: Mon→Sun (custom sort)    │ │  Shows seasonality patterns          │║
║  │                                 │ │                                      │║
║  └─────────────────────────────────┘ └──────────────────────────────────────┘║
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │            🔥 HEATMAP: HOUR vs DAY OF WEEK                             │  ║
║  │            [MATRIX VISUAL with Conditional Formatting]                 │  ║
║  │                                                                        │  ║
║  │   Rows: pbi_hourly_patterns[day_name]                                  │  ║
║  │   Columns: pbi_hourly_patterns[hour]                                   │  ║
║  │   Values: SUM(trip_count)                                              │  ║
║  │                                                                        │  ║
║  │   ┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐                 │  ║
║  │   │    │ 0  │ 1  │ 2  │... │ 8  │ 9  │... │ 17 │ 18 │                 │  ║
║  │   ├────┼────┼────┼────┼────┼────┼────┼────┼────┼────┤                 │  ║
║  │   │Mon │ ░░ │ ░░ │ ░░ │    │ ▓▓ │ ▓▓ │    │ ▓▓ │ ▓▓ │                 │  ║
║  │   │Tue │ ░░ │ ░░ │ ░░ │    │ ▓▓ │ ▓▓ │    │ ▓▓ │ ▓▓ │                 │  ║
║  │   │... │    │    │    │    │    │    │    │    │    │                 │  ║
║  │   │Sat │ ▓▓ │ ▓▓ │ ▓▓ │    │ ░░ │ ░░ │    │ ▓▓ │ ▓▓ │                 │  ║
║  │   │Sun │ ▓▓ │ ▓▓ │ ░░ │    │ ░░ │ ░░ │    │ ░░ │ ░░ │                 │  ║
║  │   └────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘                 │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

HƯỚNG DẪN TẠO HEATMAP:

1️⃣ Matrix Visual:
   • Insert → Matrix
   • Rows: day_name
   • Columns: hour
   • Values: SUM(trip_count)

2️⃣ Conditional Formatting:
   • Chọn Matrix → Format → Cell elements
   • Background color: ON → fx (conditional)
   • Format style: Gradient
   • What field: trip_count
   • Minimum: Light color (e.g., #FFFFFF)
   • Maximum: Dark color (e.g., #1E3A5F)

3️⃣ Custom Sort for Day Names:
   • Modeling → Sort by column
   • Sort day_name by day_of_week (1-7)
```


## 4.4 Page 4: Weather Impact

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    🌤️ WEATHER IMPACT ANALYSIS                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐                        ║
║  │  Clear   │ │   Rain   │ │   Snow   │ │  Severe  │                        ║
║  │  Days    │ │   Days   │ │   Days   │ │  Weather │                        ║
║  │ Avg Trips│ │ Avg Trips│ │ Avg Trips│ │ Rev Loss │                        ║
║  │  [CARD]  │ │  [CARD]  │ │  [CARD]  │ │  [CARD]  │                        ║
║  │  45.2K   │ │  38.1K   │ │  28.5K   │ │  -$2.3M  │                        ║
║  └──────────┘ └──────────┘ └──────────┘ └──────────┘                        ║
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │            📊 TRIPS BY WEATHER CONDITION                               │  ║
║  │            [CLUSTERED BAR CHART]                                       │  ║
║  │                                                                        │  ║
║  │   Y-axis: pbi_weather_impact[weather_condition]                        │  ║
║  │   X-axis: SUM(trip_count), SUM(total_revenue)                          │  ║
║  │                                                                        │  ║
║  │   Clear    ████████████████████████████████  1.2M | $15.5M             │  ║
║  │   Cloudy   ██████████████████████████        890K | $11.2M             │  ║
║  │   Rain     ████████████████████              650K | $8.5M              │  ║
║  │   Snow     ████████████                      380K | $5.2M              │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌─────────────────────────────────┐ ┌──────────────────────────────────────┐║
║  │                                 │ │                                      │║
║  │  🌡️ TEMPERATURE vs TRIPS        │ │  📈 SPEED vs WEATHER                 │║
║  │  [SCATTER PLOT]                 │ │  [LINE CHART]                        │║
║  │                                 │ │                                      │║
║  │  X: pbi_weather_impact          │ │  X: weather_condition                │║
║  │     [temp_avg]                  │ │  Y: avg_speed                        │║
║  │  Y: trip_count                  │ │                                      │║
║  │  Size: total_revenue            │ │  Shows traffic slowdown during       │║
║  │  Color: weather_condition       │ │  bad weather                         │║
║  │                                 │ │                                      │║
║  └─────────────────────────────────┘ └──────────────────────────────────────┘║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

DAX MEASURES CHO WEATHER PAGE:

// Trips on Clear Days
Clear Day Trips = 
CALCULATE(
    [Total Trips],
    pbi_weather_impact[weather_condition] = "Clear"
)

// Avg Trips per Day by Weather
Avg Daily Trips Clear = 
AVERAGEX(
    FILTER(pbi_daily_trends, pbi_daily_trends[weather_condition] = "Clear"),
    pbi_daily_trends[trip_count]
)

// Severe Weather Revenue Loss
Severe Weather Loss = 
VAR NormalRevenue = [Avg Daily Trips Clear] * [Avg Fare]
VAR SevereRevenue = 
    CALCULATE(
        AVERAGE(pbi_daily_trends[total_revenue]),
        pbi_daily_trends[is_severe_weather] = TRUE
    )
RETURN
(SevereRevenue - NormalRevenue) * 
    COUNTROWS(FILTER(pbi_daily_trends, pbi_daily_trends[is_severe_weather] = TRUE))
```


## 4.5 Page 5: Financial Analysis

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    💰 FINANCIAL PERFORMANCE                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐                        ║
║  │Revenue   │ │ YoY %    │ │Total Tips│ │Rev/Mile  │                        ║
║  │  YTD     │ │ Growth   │ │          │ │          │                        ║
║  │  [CARD]  │ │  [CARD]  │ │  [CARD]  │ │  [CARD]  │                        ║
║  │  $2.5B   │ │ ▲ +8.2%  │ │  $312M   │ │  $3.45   │                        ║
║  └──────────┘ └──────────┘ └──────────┘ └──────────┘                        ║
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │            📈 REVENUE TREND WITH FORECAST                              │  ║
║  │            [LINE CHART with Analytics]                                 │  ║
║  │                                                                        │  ║
║  │   X-axis: pbi_daily_trends[full_date]                                  │  ║
║  │   Y-axis: SUM(total_revenue)                                           │  ║
║  │                                                                        │  ║
║  │   Analytics: Trend line + Forecast (3 months)                          │  ║
║  │              Confidence interval: 95%                                  │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌─────────────────────────────────┐ ┌──────────────────────────────────────┐║
║  │                                 │ │                                      │║
║  │  🍕 REVENUE BY PAYMENT TYPE     │ │  📊 TIP % DISTRIBUTION               │║
║  │  [PIE CHART]                    │ │  [HISTOGRAM / COLUMN]                │║
║  │                                 │ │                                      │║
║  │  Legend: pbi_payment_analysis   │ │  X: Tip % bins (0-5, 5-10, etc.)     │║
║  │          [payment_type_name]    │ │  Y: Count of trips                   │║
║  │  Values: total_revenue          │ │                                      │║
║  │                                 │ │  Create bins using calculated column │║
║  │  Credit Card: 65%               │ │  or Groups feature                   │║
║  │  Cash: 30%                      │ │                                      │║
║  │  Other: 5%                      │ │                                      │║
║  │                                 │ │                                      │║
║  └─────────────────────────────────┘ └──────────────────────────────────────┘║
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │            📊 FARE DISTRIBUTION                                        │  ║
║  │            [HISTOGRAM]                                                 │  ║
║  │                                                                        │  ║
║  │   Field: gold_fact_trips[fare_amount]                                  │  ║
║  │   Bin size: $5                                                         │  ║
║  │   Filter: fare_amount between $0 and $100                              │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

HƯỚNG DẪN TẠO FORECAST:

1️⃣ Line Chart với Forecast:
   • Tạo Line chart bình thường
   • Click vào chart → Analytics pane (icon kính lúp)
   • Expand "Forecast" → Add
   • Settings:
     - Units: Months
     - Forecast length: 3
     - Ignore last: 0
     - Confidence interval: 95%
     - Style: Line with shaded area

2️⃣ Histogram:
   • Power BI không có Histogram visual mặc định
   • Option A: Dùng Column chart với Groups/Bins
   • Option B: Import "Histogram" from AppSource
   • Option C: Tạo calculated column cho bins
```


## 4.6 Page 6: ML Model Results (Bonus)

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                    🤖 ML MODEL PERFORMANCE                                   ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  Sử dụng data từ: gold_ml_model_registry, gold_ml_fare_predictions, etc.    ║
║                                                                              ║
║  ┌──────────────────┐ ┌──────────────────┐ ┌──────────────────┐             ║
║  │ 🚕 FARE MODEL    │ │ 💰 TIP MODEL     │ │ 📈 DEMAND MODEL  │             ║
║  │                  │ │                  │ │                  │             ║
║  │ RMSE: $3.45      │ │ AUC: 0.72        │ │ RMSE: 15.2       │             ║
║  │ MAE:  $2.18      │ │ Accuracy: 68%    │ │ MAE: 10.5        │             ║
║  │ R²:   0.85       │ │ F1: 0.65         │ │ R²: 0.78         │             ║
║  │                  │ │                  │ │                  │             ║
║  │ [Multi-row Card] │ │ [Multi-row Card] │ │ [Multi-row Card] │             ║
║  └──────────────────┘ └──────────────────┘ └──────────────────┘             ║
║                                                                              ║
║  ┌─────────────────────────────────┐ ┌──────────────────────────────────────┐║
║  │                                 │ │                                      │║
║  │  📊 ACTUAL vs PREDICTED         │ │  📋 MODEL VERSION HISTORY            │║
║  │  [SCATTER PLOT]                 │ │  [TABLE]                             │║
║  │                                 │ │                                      │║
║  │  From: gold_ml_fare_predictions │ │  From: gold_ml_model_registry        │║
║  │  X: actual_fare                 │ │                                      │║
║  │  Y: predicted_fare              │ │  Columns:                            │║
║  │                                 │ │  • model_name                        │║
║  │  Add reference line: y = x      │ │  • version                           │║
║  │                                 │ │  • trained_at                        │║
║  │                                 │ │  • rmse, r2                          │║
║  └─────────────────────────────────┘ └──────────────────────────────────────┘║
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                                                                        │  ║
║  │            📊 PREDICTION ERROR DISTRIBUTION                            │  ║
║  │            [HISTOGRAM]                                                 │  ║
║  │                                                                        │  ║
║  │   Field: (predicted_fare - actual_fare)                                │  ║
║  │   Should be centered around 0                                          │  ║
║  │   Bell curve = good model                                              │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
```


---
# 📘 PHẦN 5: SUMMARY & CHECKLIST

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════════════╗
║                            ✅ CHECKLIST HOÀN THÀNH                                   ║
╠══════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                      ║
║  BƯỚC 1: TẠO TABLES (Notebook này)                                                   ║
║  ────────────────────────────────                                                    ║
║  □ Chạy tất cả cells trong notebook này                                              ║
║  □ Verify 6 pbi_* tables được tạo                                                    ║
║                                                                                      ║
║  BƯỚC 2: TẠO SEMANTIC MODEL                                                          ║
║  ─────────────────────────────                                                       ║
║  □ Mở Lakehouse trong Fabric                                                         ║
║  □ Click "New semantic model"                                                        ║
║  □ Chọn gold_* và pbi_* tables                                                       ║
║  □ Click Confirm                                                                     ║
║                                                                                      ║
║  BƯỚC 3: CẤU HÌNH RELATIONSHIPS                                                      ║
║  ──────────────────────────────                                                      ║
║  □ Mở Model view                                                                     ║
║  □ Tạo 8 relationships (xem Section 2.3)                                             ║
║  □ Set dropoff_location relationship = Inactive                                      ║
║  □ Verify tất cả relationships hiển thị đúng                                         ║
║                                                                                      ║
║  BƯỚC 4: TẠO MEASURES                                                                ║
║  ────────────────────                                                                ║
║  □ Tạo _Measures table                                                               ║
║  □ Copy và tạo Basic measures (Section 3.2)                                          ║
║  □ Copy và tạo Average measures (Section 3.3)                                        ║
║  □ Copy và tạo Time Intelligence measures (Section 3.4)                              ║
║  □ Copy và tạo Conditional measures (Section 3.5)                                    ║
║  □ Format measures với đơn vị phù hợp                                                ║
║                                                                                      ║
║  BƯỚC 5: TẠO REPORT PAGES                                                            ║
║  ───────────────────────                                                             ║
║  □ Page 1: Executive Dashboard                                                       ║
║  □ Page 2: Geographic Analysis                                                       ║
║  □ Page 3: Time Analysis (với Heatmap)                                               ║
║  □ Page 4: Weather Impact                                                            ║
║  □ Page 5: Financial Analysis (với Forecast)                                         ║
║  □ Page 6: ML Results (optional)                                                     ║
║                                                                                      ║
║  BƯỚC 6: FINISHING TOUCHES                                                           ║
║  ────────────────────────                                                            ║
║  □ Add Slicers (Year, Month, Borough)                                                ║
║  □ Sync slicers across pages                                                         ║
║  □ Apply consistent color theme                                                      ║
║  □ Add report title và page titles                                                   ║
║  □ Publish to Fabric workspace                                                       ║
║                                                                                      ║
╚══════════════════════════════════════════════════════════════════════════════════════╝
""")

## Final Summary

In [ ]:
print("=" * 80)
print("📊 POWER BI INTEGRATION COMPLETE")
print("=" * 80)

# Count tables
total_tables = 0
for table in ["pbi_daily_trends", "pbi_hourly_patterns", "pbi_location_analytics", 
              "pbi_weather_impact", "pbi_payment_analysis", "pbi_borough_monthly"]:
    try:
        spark.table(table)
        total_tables += 1
    except:
        pass

print(f"""
┌────────────────────────────────────────────────────────────────────────────────┐
│ TABLES CREATED:           {total_tables}/6 tables                                         │
├────────────────────────────────────────────────────────────────────────────────┤
│ RELATIONSHIPS TO CREATE:  8 relationships                                      │
│ DAX MEASURES PROVIDED:    42 measures                                          │
│ REPORT PAGES DESIGNED:    6 pages                                              │
├────────────────────────────────────────────────────────────────────────────────┤
│ NEXT STEPS:                                                                    │
│                                                                                │
│ 1️⃣  Vào Fabric → Lakehouse → "New semantic model"                              │
│ 2️⃣  Chọn tables: gold_* và pbi_*                                               │
│ 3️⃣  Mở Model view → Tạo 8 relationships                                        │
│ 4️⃣  Tạo measures từ DAX library ở trên                                         │
│ 5️⃣  Click "New report" → Build 6 pages                                         │
│ 6️⃣  Publish và share!                                                          │
└────────────────────────────────────────────────────────────────────────────────┘
""")
print("=" * 80)